In [1]:
# Importar las bibliotecas necesarias
from sklearn import svm, metrics
from skimage.feature import hog
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import csv
import concurrent.futures
from sklearn.model_selection import StratifiedKFold

path = '../data'
categorias = ['NORMAL', 'PNEUMONIA']

total_data = []

def procesar_imagen(imagen_path, size, etiqueta):
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f'No se pudo leer la imagen: {imagen_path}')
        return None, None

    imagen = cv2.resize(imagen, size)
    fd = hog(imagen, orientations=8, pixels_per_cell=(8, 8),
             cells_per_block=(1, 1), visualize=False)

    return fd, etiqueta

def cargar_datos(ruta, size=(128, 128)):
    datos = []
    etiquetas = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i, cat in enumerate(categorias):
            carpeta = join(ruta, cat)
            archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
            for archivo in archivos:
                imagen_path = join(carpeta, archivo)
                futures.append(executor.submit(procesar_imagen, imagen_path, size, i))

        for future in concurrent.futures.as_completed(futures):
            fd, label = future.result()
            if fd is not None:
                datos.append(fd)
                etiquetas.append(label)

    if len(datos) == 0 or len(etiquetas) == 0:
        raise ValueError("No se pudieron cargar los datos. Asegúrate de que la ruta y las categorías son correctas.")

    return np.array(datos), np.array(etiquetas)



# Cargar datos de entrenamiento, validación y prueba
datos, etiquetas = cargar_datos(join(path, 'train'))

# Asegurarse de que hay datos para al menos dos clases
if len(np.unique(etiquetas)) < 2:
    raise ValueError("Se debe cargar datos para al menos dos clases en cada conjunto de datos.")

cv = StratifiedKFold(n_splits=5)

#nombre_archivo_csv = '/content/drive/MyDrive/Proyecto VAI-AAI/first_version/cv_svm_hog.csv'
nombre_archivo_csv = '/content/drive/MyDrive/Estudios/Máster IIR/Segundo cuatrimestre/Proyecto VAI-AAI/first_version/cv_svm_hog.csv'

# Preparar archivo CSV para guardar los resultados
with open(nombre_archivo_csv, 'a', newline='') as archivo_csv:
    es_vacio = archivo_csv.tell() == 0
    fieldnames = ['Fold', 'Exactitud', 'Sensibilidad', 'Especificidad', 'Precisión', 'F1-Score', 'AUC']
    writer = csv.DictWriter(archivo_csv, fieldnames=fieldnames)
    if es_vacio:
      writer.writeheader()

    for fold, (train_index, val_index) in enumerate(cv.split(datos, etiquetas)):
        datos_entrenamiento, datos_prueba = datos[train_index], datos[val_index]
        etiquetas_entrenamiento, etiquetas_prueba = etiquetas[train_index], etiquetas[val_index]

        # Crear y entrenar el modelo SVM
        modelo = svm.SVC(gamma='scale', probability=True)  # Asegúrate de activar probability=True para ROC AUC
        modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)

        # Evaluar el modelo
        etiquetas_predichas = modelo.predict(datos_prueba)
        exactitud = metrics.accuracy_score(etiquetas_prueba, etiquetas_predichas)
        cm = confusion_matrix(etiquetas_prueba, etiquetas_predichas)
        sensibilidad = recall_score(etiquetas_prueba, etiquetas_predichas)
        especificidad = cm[0,0] / (cm[0,0] + cm[0,1])
        precision = precision_score(etiquetas_prueba, etiquetas_predichas)
        f1 = f1_score(etiquetas_prueba, etiquetas_predichas)

        # Calcular AUC
        if hasattr(modelo, "decision_function"):
            scores = modelo.decision_function(datos_prueba)
        else:
            scores = modelo.predict_proba(datos_prueba)[:, 1]
        auc = roc_auc_score(etiquetas_prueba, scores)

        # Escribir resultados en el archivo CSV
        writer.writerow({
            'Fold': fold + 1,
            'Exactitud': exactitud * 100,
            'Sensibilidad': sensibilidad * 100,
            'Especificidad': especificidad * 100,
            'Precisión': precision * 100,
            'F1-Score': f1,
            'AUC': auc
        })

# Cargar datos de prueba
datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'))

# Predecir etiquetas para el conjunto de prueba
etiquetas_predichas_prueba = modelo.predict(datos_prueba)

# Calcular y mostrar métricas de rendimiento para el conjunto de prueba
exactitud_prueba = metrics.accuracy_score(etiquetas_prueba, etiquetas_predichas_prueba)
cm_prueba = confusion_matrix(etiquetas_prueba, etiquetas_predichas_prueba)
sensibilidad_prueba = recall_score(etiquetas_prueba, etiquetas_predichas_prueba)
especificidad_prueba = cm_prueba[0,0] / (cm_prueba[0,0] + cm_prueba[0,1])
precision_prueba = precision_score(etiquetas_prueba, etiquetas_predichas_prueba)
f1_prueba = f1_score(etiquetas_prueba, etiquetas_predichas_prueba)

# Calcular AUC para el conjunto de prueba
if hasattr(modelo, "decision_function"):
    scores_prueba = modelo.decision_function(datos_prueba)
else:
    scores_prueba = modelo.predict_proba(datos_prueba)[:, 1]
auc_prueba = roc_auc_score(etiquetas_prueba, scores_prueba)

with open(nombre_archivo_csv, 'a', newline='') as archivo_csv:  # 'a' para añadir al archivo existente
    fieldnames = ['Fold', 'Exactitud', 'Sensibilidad', 'Especificidad', 'Precisión', 'F1-Score', 'AUC']
    writer = csv.DictWriter(archivo_csv, fieldnames=fieldnames)

    # No necesitamos volver a escribir el encabezado, asumiendo que estamos añadiendo al archivo existente

    # Escribir resultados del conjunto de prueba en el archivo CSV
    writer.writerow({
        'Fold': 'Test',
        'Exactitud': exactitud_prueba * 100,
        'Sensibilidad': sensibilidad_prueba * 100,
        'Especificidad': especificidad_prueba * 100,
        'Precisión': precision_prueba * 100,
        'F1-Score': f1_prueba,
        'AUC': auc_prueba
    })

Mounted at /content/drive
